In [1]:
import numpy as np 
import pandas as pd 
import os
import re
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


In [2]:
df = pd.read_csv("election.csv")

In [3]:
import re
emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
stop_words.add("rt")
def myProcess(twitterText):
    # this regex should be created once, so make it as global variable
    # remove URLs
    nopunc = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', twitterText)
    nopunc = re.sub(r'http\S+', '', nopunc)
    #remove emoji's
    twitterText = emoji.sub(r'', twitterText)
    # expand abbreviations
    text = twitterText.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)        
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text) 
    text = re.sub(r"\'ll", " will", text)  
    text = re.sub(r"\'ve", " have", text)  
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"did't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    twitterText = re.sub(r"have't", "have not", text)
    
    #nopunc = re.sub("'s", '', nopunc)
    # remove usernames
    nopunc = re.sub('@[^\s]+', '', nopunc)
    # remove the # in #hashtag
    nopunc = re.sub(r'#([^\s]+)', r'\1', nopunc)
    # remove punctuation
    #nopunc = regex.sub('', nopunc).lower()
    # npunc = re.sub(r"(.)\1{1,}",r"\1",nopunc)
    pattern = re.compile(r"(.)\1{2,}")
    nopunc = pattern.sub(r"\1\1", nopunc).lower()
    # nopunc = re.sub(r'([a-z])\1+', r'\1', nopunc)
    #nopunc = re.sub(r'[^A-Za-z. ]', '', nopunc)
    nopunc = re.sub("[^A-Za-z ]","",nopunc)
    nopunc = re.sub("semst","",nopunc)
    nopunc = re.sub(r'  ',' ',nopunc)
    return [i for i in nopunc.split() if i not in stop_words]

In [4]:
df

,Unnamed: 0,tweet_id,tweet_text,date,annotations,anot
0,0,509464992404357120,@kyool I can respect that.,Tue Sep 09 22:14:54 +0000 2014,"[{'annotator': 'annotator00', 'annotation': 'n...","{'care', 'loyalty', 'fairness', 'non-moral'}"
1,1,798700545364135942,RT @ShitBsbPlyrsSay: The disrespect 😂💀\n https...,Wed Nov 16 01:33:43 +0000 2016,"[{'annotator': 'annotator00', 'annotation': 'n...","{'betrayal', 'subversion', 'non-moral'}"
2,2,782348427589931008,Our walls are thin so @tyler_geordon makes me ...,Sat Oct 01 22:36:15 +0000 2016,"[{'annotator': 'annotator00', 'annotation': 'n...",{'non-moral'}
3,3,835619770322026496,RT @Jenyour: Lord pls teach us \nTo love those...,Sat Feb 25 22:37:32 +0000 2017,"[{'annotator': 'annotator00', 'annotation': 'p...","{'purity', 'care', 'fairness'}"
4,4,720947449976778752,Fear of the Lord teaches a person wisdom and h...,Fri Apr 15 12:10:41 +0000 2016,"[{'annotator': 'annotator00', 'annotation': 'p...","{'purity', 'authority'}"
5,5,577360913101033472,74-yr-old NUN gang raped in #India. How sick a...,Mon Mar 16 06:49:03 +0000 2015,"[{'annotator': 'annotator00', 'annotation': 'd...","{'harm', 'degradation'}"
6,6,833206725801033728,"RT @XakSullivan: @janlovesword \n\n""The foolis...",Sun Feb 19 06:48:58 +0000 2017,"[{'annotator': 'annotator00', 'annotation': 'p...","{'harm', 'purity', 'cheating', 'non-moral'}"
7,7,585481467289010176,RT @KATMOVEMENTS: Is red my color?💋 #justforfu...,Tue Apr 07 16:37:14 +0000 2015,"[{'annotator': 'annotator00', 'annotation': 'n...",{'non-moral'}
8,8,837493791082737664,But anyone who might admit to hypocrisy or dou...,Fri Mar 03 02:44:14 +0000 2017,"[{'annotator': 'annotator02', 'annotation': 'n...","{'cheating', 'fairness', 'non-moral'}"
9,9,838926720736477184,The chief value of Hinduism is the belief that...,Tue Mar 07 01:38:11 +0000 2017,"[{'annotator': 'annotator00', 'annotation': 'p...","{'purity', 'loyalty', 'care'}"


In [5]:
df["clean"] = df.tweet_text.apply(myProcess)

In [6]:
df

,Unnamed: 0,tweet_id,tweet_text,date,annotations,anot,clean
0,0,509464992404357120,@kyool I can respect that.,Tue Sep 09 22:14:54 +0000 2014,"[{'annotator': 'annotator00', 'annotation': 'n...","{'care', 'loyalty', 'fairness', 'non-moral'}",[respect]
1,1,798700545364135942,RT @ShitBsbPlyrsSay: The disrespect 😂💀\n https...,Wed Nov 16 01:33:43 +0000 2016,"[{'annotator': 'annotator00', 'annotation': 'n...","{'betrayal', 'subversion', 'non-moral'}",[disrespect]
2,2,782348427589931008,Our walls are thin so @tyler_geordon makes me ...,Sat Oct 01 22:36:15 +0000 2016,"[{'annotator': 'annotator00', 'annotation': 'n...",{'non-moral'},"[walls, thin, makes, play, music, poops]"
3,3,835619770322026496,RT @Jenyour: Lord pls teach us \nTo love those...,Sat Feb 25 22:37:32 +0000 2017,"[{'annotator': 'annotator00', 'annotation': 'p...","{'purity', 'care', 'fairness'}","[lord, pls, teach, us, love, hate, usto, pray,..."
4,4,720947449976778752,Fear of the Lord teaches a person wisdom and h...,Fri Apr 15 12:10:41 +0000 2016,"[{'annotator': 'annotator00', 'annotation': 'p...","{'purity', 'authority'}","[fear, lord, teaches, person, wisdom, humility..."
5,5,577360913101033472,74-yr-old NUN gang raped in #India. How sick a...,Mon Mar 16 06:49:03 +0000 2015,"[{'annotator': 'annotator00', 'annotation': 'd...","{'harm', 'degradation'}","[yrold, nun, gang, raped, india, sick, disgust..."
6,6,833206725801033728,"RT @XakSullivan: @janlovesword \n\n""The foolis...",Sun Feb 19 06:48:58 +0000 2017,"[{'annotator': 'annotator00', 'annotation': 'p...","{'harm', 'purity', 'cheating', 'non-moral'}","[foolish, shall, stand, thy, sight, thou, hate..."
7,7,585481467289010176,RT @KATMOVEMENTS: Is red my color?💋 #justforfu...,Tue Apr 07 16:37:14 +0000 2015,"[{'annotator': 'annotator00', 'annotation': 'n...",{'non-moral'},"[red, color, justforfun, redlingerie, playboym..."
8,8,837493791082737664,But anyone who might admit to hypocrisy or dou...,Fri Mar 03 02:44:14 +0000 2017,"[{'annotator': 'annotator02', 'annotation': 'n...","{'cheating', 'fairness', 'non-moral'}","[anyone, might, admit, hypocrisy, double, stan..."
9,9,838926720736477184,The chief value of Hinduism is the belief that...,Tue Mar 07 01:38:11 +0000 2017,"[{'annotator': 'annotator00', 'annotation': 'p...","{'purity', 'loyalty', 'care'}","[chief, value, hinduism, belief, life, one, co..."


In [7]:
convertDict = {
    1:"Care", 2:"Harm",
    3:"Fairness", 4:"Cheating",
    5:"Loyalty",6:"Betrayal",
    7:"Authority",8:"Subversion",
    9:"Purity",10:"Degradation"
}



In [8]:
posSent = set([v.lower() for k,v in convertDict.items() if k%2!=0])
negSent = set([v.lower() for k,v in convertDict.items() if k%2==0])

In [9]:
posSent

{'authority', 'care', 'fairness', 'loyalty', 'purity'}

In [10]:
negSent

{'betrayal', 'cheating', 'degradation', 'harm', 'subversion'}

In [11]:
from EnhDict2 import enhDict2
posWord = set()
negWord = set()
for k,v in enhDict2.items():
    if len(v) == 0:
        continue
    currentValue = v[0]
    if currentValue%2 == 0:
        negWord.add(k)
    else:
        posWord.add(k)

In [12]:
trainDataset = []
trainLabel = []
df = df.fillna("")
for i in range(len(df)):
    currentData = [0,0]
    curdf = df.iloc[i]
    curdata = curdf.clean
    curStance = [i.strip().lower() for i in curdf.anot[1:-1].replace("'","").split(",")]
    for each in curdata:
            if each in posWord:
                currentData[0]+=1
            elif each in negWord:
                currentData[1]+=1
    for eachStance in curStance:
        if eachStance in posSent:
            curLabel = 0
        else:
            curLabel = 1
        trainDataset.append(currentData)
        trainLabel.append(curLabel)
        

In [13]:
[i.strip() for i in df.iloc[0].anot[1:-1].replace("'","").split(",")]

['care', 'loyalty', 'fairness', 'non-moral']

In [14]:
X,Y = trainDataset,trainLabel

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.9,random_state=23)

In [16]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
np.argwhere(y_train==0)

array([[   0],
       [   2],
       [   3],
       ...,
       [8605],
       [8608],
       [8609]], dtype=int64)

In [17]:
clf = SVC()

In [18]:
clf.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [19]:
y_pred = clf.predict(x_test)

In [20]:
accuracy_score(y_test,y_pred,2)

0.7053291536050157

In [21]:
import re

with open("biden.txt","r",encoding="utf-8") as fd:
    allData = fd.read()
nData = re.sub(r"\n"," ",allData)
nData = re.sub(r"[^A-Za-z. ]","",nData)

In [22]:
x = [i.strip() for i in nData.split(".")]

In [23]:
couter = 0
allData = []
while couter < len(x):
    allData.append(" ".join(x[couter:couter+4]))
    couter+=4

In [24]:
allData

['Please welcome back to the stage Lady Gaga Please welcome business agent for Laborers Union Local  John Duffy Whoa Hello Pittsburgh',
 'Im Jill Bidens husband and Lady Gagas friend We go back a while One thing I wasnt going to say but Ill said now she spent a lot of time helping me with the Violence Against Women Act and helping so many women who needed help God love you youre wonderful',
 'Thank you thank you thank you Folks John thanks old buddy for sharing your story And thanks to Mr',
 'Mayor I dont know where you are I cant see out here but Ill tell you what thanks for the passport into town And thanks for all the speakers we heard tonight as well as the incredible as I said Lady Gaga I dont know whether  You couldnt see but all my grandkids are over there Lady Gaga',
 'And when you were singing knew every word of every song They were jumping and moving I got to get a picture with them Anyway',
 'My buddy Congressman Mike Doyle Governor Tom Wolf thank you for standing with me an

In [25]:
trainData = []
for i in allData:
    currdata = [0,0]
    cur = [w for w in i.split() if w not in stop_words]
    for j in cur:
        if j in posWord:
            currdata[0]+=1
        elif j in negWord:
            currdata[1]+=1
    trainData.append(currdata)
            

In [26]:
y_pred = clf.predict(trainData)

In [27]:
y_pred

array([1, 0, 1, ..., 0, 1, 1])

In [28]:
np.sum(y_pred)/len(y_pred)

0.6709511568123393